<a href="https://colab.research.google.com/github/sachaRfd/Sentiment-Analysis-NLP/blob/main/Sentiment_Analysis_Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Simple Sentiment Analysis using IMDB PyTorch Dataset and simple LSTM:

All Imports:

In [1]:
!pip install torchdata  # Install Torch Datasets
!pip install nltk  # Import the Natural Language Toolkit --> Most Common

import nltk  # Download key files
nltk.download('punkt')  # Sequence Tokeniser
nltk.download('stopwords')  # List of Most Common StopWords
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import string 



import numpy as np
import pandas as pd

import gc
from tqdm import tqdm

import torch
from torch import nn
from torch.nn.functional import pad
import torch.nn.functional as F
from torchtext.data import to_map_style_dataset
from torch.utils.data import DataLoader
from torch.optim import RMSprop


# Set Device to GPU is available - otherwise set to CPU: 
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Your Current Device is {device}')  # Check the Colab Device we are using

from torchtext import data, datasets  # Import the datasets
from sklearn.model_selection import train_test_split  # Import splitting function
from sklearn.metrics import accuracy_score
import torchdata

from torchtext.vocab import GloVe  # Import the Glove Embedding

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.6 MB 4.1 MB/s 
     |██████████████████████████████  | 834.1 MB 1.2 MB/s eta 0:00:45tcmalloc: large alloc 1147494400 bytes == 0x391f4000 @  0x7f874b53b615 0x5d6f4c 0x51edd1 0x51ef5b 0x4f750a 0x4997a2 0x4fd8b5 0x4997c7 0x4fd8b5 0x49abe4 0x4f5fe9 0x55e146 0x4f5fe9 0x55e146 0x4f5fe9 0x55e146 0x5d8868 0x5da092 0x587116 0x5d8d8c 0x55dc1e 0x55cd91 0x5d8941 0x49abe4 0x55cd91 0x5d8941 0x4990ca 0x5d8868 0x4997a2 0x4fd8b5 0x49abe4
     |████████████████████████████████| 887.4 MB 1.7 kB/s 
     |████████████████████████████████| 140 kB 60.0 MB/s 
     |████████████████████████████████| 317.1 MB 33 kB/s 
     |████████████████████████████████| 849 kB 52.1 MB/s 
     |████████████████████████████████| 21.0 MB 1.2 MB/s 
     |████████████████████████████████| 557.1 MB 12 kB/s 
     |████████████████████████████████| 127 kB 48.1 MB/s 
  Attempting uninstall: url

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


Your Current Device is cpu


Let's get the Train, Validation and Training Sets ready: 

In [38]:
# Get the train and test splits form the IMDB Dataset
train_dataset, test_dataset  = datasets.IMDB(root = '.data', split = ('train', 'test'))

# Let's now split the test set into a test and validation set: 
test_dataset, valid_dataset = train_test_split(list(test_dataset), train_size=.8)


## Understanding the Dataset: 
IMDB Reviews


In [39]:
print(f'The shape of the Train set is {len(list(train_dataset))}')  #  Have to Convert to List
print(f'The shape of the Validation set is {len(valid_dataset)}')
print(f'The shape of the Test set is {len(test_dataset)}')

The shape of the Train set is 25000
The shape of the Validation set is 5000
The shape of the Test set is 20000


Shape of the training Dataset is 25_000. 

Let's Check if our data is balanced in the training set: 

In [53]:
# Code to check for balanced dataset
dataframe_ = pd.DataFrame(list(train_dataset), columns=['Y', 'x'])
dataframe_.Y.value_counts()

1    12500
2    12500
Name: Y, dtype: int64

### Let's now visualise some of the reviews: 

In [44]:
# Plotting the First 2 Reviews
list(train_dataset)[:5]

[(1,
  'I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered pornographic. Really, the sex and nudity scenes are few and far betw

## To summarise the dataset: 
- The dataset consists of Movie reviews taken from IMDB
- The train set is formed of 25_000 reviews
- The validation set of 5_000 reviews
- and the Test set is of 20_000 reviews. 
- In the Y variable, a 1 consists of a Negative Review and 2 a Positive Review
- We can also see that our dataset is BALANCED, with 12_500 bad reviews and 12_5000 good reviews.

# Data Preprocessing: 

For simlpe NLP applications, the data has to be processed in a certain manner: 
- Things to Check For:
  - All lower-case text
  - No Numbers in text
  - No Punctutation - Good for generalisation - eventhough some people use punctuation to show sentiment
- Transformation of the sentences into list of tokens - Therefore the sentence becomes a list of words
- We have to tokenise the words - We will be using words from the GloVe library.
  - We want to get the Index of our words in the GloVe library.
- Padding of the sentences is also required as some of the reviews are very long or relatively short. Let's use a maximum padding of 150 here - For no reason.

# List of Functions: 
- To Remove Numbers in the sentences
- To Remove Punctuation
- To Tokenise the sentences
- To Remove Unwanted Stopwords
- To Get the Index of the words in the GloVe library
- To Pad the Sentences
- A final Function which transforms the inputted test by using all the above functions and converting the sentences to lowercase.

In [56]:
def remove_numbers(text):
  '''Function to Remove Numbers from inputted text'''
  text = ''.join(word for word in text if not word.isdigit())
  return text

def remove_punctuation(text):
  '''Function to Remove all Punctuation from inputted text'''

  for punctuation in string.punctuation:
     text = text.replace(punctuation, '')  # Replace the Punctuation with empty space
  return text

def tokenize(text):
  '''Function to Tokenise any inputted text using NLTK tokenise'''

  word_tokens = word_tokenize(text)  # Tokenise Using the NLTK Tokenise Function
  return word_tokens

def remove_stopwords(word_tokens, language='english'):
  '''Function to remove all stopwords in given language from the inputted words tokens'''

  stop_words = set(stopwords.words(language))  # Most common English Stopwords
  word_tokens = [w for w in word_tokens if not w in stop_words]  # Get list of words if they are not stopwords
  return word_tokens

glove = GloVe(dim='50', name='6B', max_vectors=20000)  # Get the Glove with 50 dimension vector with a vocabulary size of 20_000

def get_index(text, vocab=glove):
  '''Function that gets the index of each token in a text from the GloVe Library'''

  embedded_text = []
  for word in text:
     try:
         embedded_text.append(glove.stoi[word])  # Get String to Integer
     except:
         pass
  return embedded_text  # return list of the indices of the tokenised words in the GloVe library


def pad_sentence(text, MAX_LENGTH = 150):
  ''' Function that Pads a sentence to a given length'''

  if text.shape[0]>=MAX_LENGTH:
      return text[:MAX_LENGTH]
  else:
      return pad(text, (0, MAX_LENGTH-text.shape[0]), 'constant',0).long()


# Final Transform Function: 

def transform_text(text):
  '''Function that applies all the Data-Preprocessing Functions'''
  
  text = text.lower()
  text = remove_numbers(text)
  text = remove_punctuation(text)
  text = tokenize(text)
  text = remove_stopwords(text)
  text = torch.tensor(get_index(text)).long()
  return pad_sentence(text)

.vector_cache/glove.6B.zip: 862MB [02:40, 5.36MB/s]                           
100%|█████████▉| 19999/20000 [00:00<00:00, 36224.05it/s]


Now that we have setup our data-preprocessing, let's test it out on an example from our training dataset: 

In [62]:
example_train = list(train_dataset)[5][1]  # Get a random train data
transform_text(example_train)

tensor([   54,   339,   220,   674,  1588,  2891,  8560,  1588,   978,  1607,
          921,  2001, 12073,   117,  2837,   219,  1739, 11184, 10487,   122,
          151, 12425,   175,  2782,  1378,  6959,   152,   164,  9797,  4629,
          364,   319,  1607,  1380,   570,  6801,  5412,   521,   298,  3468,
         1254,   492,  1797,  7582,   151,  1507,   978,    69,   580,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0, 

Now that our pre-processing function seem to work as we want - Let's finalise the dataset with a dataloader. 

To not apply our transform function to all the data at once, let'd just apply it batch by batch from the dataloader using transform_batch function: 

In [69]:
train_y = torch.tensor([item[0] for item in list(train_dataset)])-1
train_x = torch.stack([transform_text(item[1]) for item in list(train_dataset)])

val_y = torch.tensor([item[0] for item in list(test_dataset)])-1
val_x = torch.stack([transform_text(item[1]) for item in list(test_dataset)])

test_y = torch.tensor([item[0] for item in list(test_dataset)])-1
test_x = torch.stack([transform_text(item[1]) for item in list(test_dataset)])

In [70]:
def transform_batch(batch):
    Y, X = list(zip(*batch))
    
    X_embedded = torch.stack([transform_text(txt) for txt in X])  # Get the transformed - embedded text
    
    return X_embedded, torch.tensor(Y).long()-1  # Return the Embedded text and the Y variable as .long() as it is a categorical label

train_dataset=  to_map_style_dataset(train_dataset)  # We will be using the to_map_style_dataset as it CHECK WHAT IT DOES

train_loader = DataLoader(train_dataset, batch_size=256, collate_fn=transform_batch, shuffle=True)  # Make sure to have shuffle on true for best training

In [64]:
# Checking the batch sizes
# for X, Y in train_loader:
#     print(X.shape, Y.shape)
#     break

torch.Size([256, 150]) torch.Size([256])


Let's use the GloVe vectors in our embedding layer, where we inputted 50 dimensions for each vector and a vocabulary size of 20_000.

Here is an implementation of a embedded layer taken from my lecture in DL:

In [71]:
def create_emb_layer(weights_matrix, non_trainable=True):
    num_embeddings, embedding_dim = weights_matrix.size()
    emb_layer = nn.Embedding(num_embeddings, embedding_dim,padding_idx=0)
    emb_layer.load_state_dict({'weight': weights_matrix})
    if non_trainable:
        emb_layer.weight.requires_grad = False

    return emb_layer, num_embeddings, embedding_dim

## Now that we have all of the preprocessing, the embedding, and the data loaders ready, we can start thinking of Deep Learning Models.

In [172]:
class LSTM(nn.Module):
    def __init__(self, hid_dim, output_dim, num_layers = 2):
        super(LSTM, self).__init__()
        
        self.hid_dim = hid_dim
        
        self.embedding, num_embeddings, embedding_dim = create_emb_layer(glove.vectors, False)
        
        n_layers = num_layers

        self.lstm = nn.LSTM(embedding_dim, hid_dim, n_layers,dropout=0, batch_first=True)
        self.linear = nn.Linear(hid_dim,100)
        self.relu = nn.ReLU()
        self.fc = nn.Linear(100, output_dim)
        self.dropout = nn.Dropout(.5)
        
        self.reset_parameters()
        
    def reset_parameters(self):
        std= 1.0 / np.sqrt(self.hid_dim)
        
        for w in self.parameters():
            w.data.uniform_(-std, std)
        

    def forward(self, text):

        embedded = self.embedding(text)


        batch_size, seq_len,  _ = embedded.shape
        hid_dim = self.lstm.hidden_size
            
        outputs, (hidden, cell) = self.lstm(embedded)

        outputs = outputs[:, -1]
        
        prediction = self.fc(self.dropout(self.relu(self.linear(outputs))))


        return prediction

In [153]:
def CalcValLossAndAccuracy(model, loss_fn, val_X, val_Y):
    
    #print(f'Calculating Epoch Loss and Accuracy:')
    
    losses = []
    accuracies = []
    model.eval()
    with torch.no_grad():
        X, Y, title = (val_x, val_y,'Validation')
        X = val_X.to(device)
        Y = val_Y.to(device)
            
        outputs = model(X).squeeze()
        loss = loss_fn(outputs, Y.float())
            
        preds = [1 if p>=.5 else 0 for p in torch.sigmoid(outputs)]
        accuracy = accuracy_score(Y.detach().cpu().numpy().tolist(),preds)
            
        accuracies.append(accuracy)
        losses.append(loss)

        
        print(f'{title} Loss : {loss:.3f}')
        print(f"{title} Accuracy  : {accuracy:.3f}")
    
    return losses, accuracies


def TrainModel(model, loss_fn, optimizer, train_loader, epochs=10):
    train_losses = []
    train_accuracy = []
    val_losses = []
    val_accuracy = []
    
    for i in range(1, epochs+1):
        
        print('-'*100)
        print(f'EPOCH {i}')
        print('-'*100)
        
        epoch_losses = []

        model.train()

        
        for X, Y in tqdm(train_loader, colour='BLUE'):

            X = X.to(device)
            Y = Y.to(device)
            
            Y_preds = model(X).squeeze()
            loss = loss_fn(Y_preds, Y.float())
            
            epoch_losses.append(loss.item())

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        print("Train Loss : {:.3f}".format(torch.tensor(epoch_losses).mean()))
        
        losses, acc = CalcValLossAndAccuracy(model, loss_fn, X, Y)
        train_losses.append(losses[0])
        train_accuracy.append(losses[0])
        val_losses.append(acc[0])
        val_accuracy.append(acc[0])
        
    return train_losses, val_losses, train_accuracy, val_accuracy

def evaluateModel(model, loss_fn, x, Y):

  model.eval()
  with torch.no_grad():
    x = x.to(device)
    Y = Y.to(device)

    outputs = model(x).squeeze()

    preds = [1 if p>=.5 else 0 for p in torch.sigmoid(outputs)]
    accuracy = accuracy_score(Y.detach().cpu().numpy().tolist(),preds)

  return accuracy

In [140]:

epochs = 2
learning_rate = 1e-3

loss_fn = nn.BCEWithLogitsLoss()
text_classifier_2_epochs = LSTM(20,1).to(device)

optimizer = RMSprop(text_classifier_2_epochs.parameters(), lr=learning_rate)

print("STARTING TRAINING")
print("MODEL ARCHITECTURE:")
print(text_classifier_2_epochs)
print(" ")


TrainModel(text_classifier_2_epochs, loss_fn, optimizer, train_loader, epochs)

STARTING TRAINING
MODEL ARCHITECTURE:
LSTM(
  (embedding): Embedding(20000, 50, padding_idx=0)
  (lstm): LSTM(50, 20, num_layers=2, batch_first=True)
  (linear): Linear(in_features=20, out_features=100, bias=True)
  (relu): ReLU()
  (fc): Linear(in_features=100, out_features=1, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
)
 
----------------------------------------------------------------------------------------------------
EPOCH 1
----------------------------------------------------------------------------------------------------


100%|██████████| 98/98 [01:17<00:00,  1.27it/s]


Train Loss : 0.694
Validation Loss : 0.693
Validation Accuracy  : 0.476
----------------------------------------------------------------------------------------------------
EPOCH 2
----------------------------------------------------------------------------------------------------


100%|██████████| 98/98 [01:13<00:00,  1.34it/s]

Train Loss : 0.684
Validation Loss : 0.660
Validation Accuracy  : 0.548


([tensor(0.6934), tensor(0.6603)],
 [0.47619047619047616, 0.5476190476190477],
 [tensor(0.6934), tensor(0.6603)],
 [0.47619047619047616, 0.5476190476190477])

Let's Checkout the model we have trained: 

In [131]:
test_dataset[10]

(1,
 "After seeing this film I felt sick to my stomach and if I had seen one more minute I would have had to rush to the bathroom and vomit til dawn. A sick film that was NOT funny and was NOT worth the money, any money at all. If anybody ever wants to see this movie don't! Your kids will never forgive you and will claim sickness for a week. So if you value your child's education and want to stimulate your child's mind please don't see this movie. I beg of you, DON'T!")

In [141]:
text_classifier_2_epochs.eval()
with torch.no_grad():
    print('Predicted values with only 2 epochs: ')
    print(torch.sigmoid(text_classifier_2_epochs(test_x[:5].to(device))))
    print(test_y[:5])
    print(f'The Accuracy of our model with 2 epochs is of {evaluateModel(text_classifier_2_epochs, loss_fn, test_x, test_y)}')

Predicted values with only 2 epochs: 
tensor([[0.4711],
        [0.4711],
        [0.8088],
        [0.4711],
        [0.4711]])
tensor([0, 0, 1, 0, 0])
The Accuracy of our model with 2 epochs is of 0.53535


# Let's now try training the model with 10 epochs:

In [134]:
epochs = 10
learning_rate = 1e-3


loss_fn = nn.BCEWithLogitsLoss()
text_classifier_10_epochs = LSTM(20,1).to(device)

optimizer = RMSprop(text_classifier_10_epochs.parameters(), lr=learning_rate)

print("STARTING TRAINING")
print("MODEL ARCHITECTURE:")
print(text_classifier_10_epochs)
print(" ")


TrainModel(text_classifier_10_epochs, loss_fn, optimizer, train_loader, epochs)

STARTING TRAINING
MODEL ARCHITECTURE:
LSTM(
  (embedding): Embedding(20000, 50, padding_idx=0)
  (lstm): LSTM(50, 20, num_layers=2, batch_first=True)
  (linear): Linear(in_features=20, out_features=100, bias=True)
  (relu): ReLU()
  (fc): Linear(in_features=100, out_features=1, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
)
 
----------------------------------------------------------------------------------------------------
EPOCH 1
----------------------------------------------------------------------------------------------------


100%|██████████| 98/98 [01:16<00:00,  1.29it/s]


Train Loss : 0.695
Validation Loss : 0.689
Validation Accuracy  : 0.554
----------------------------------------------------------------------------------------------------
EPOCH 2
----------------------------------------------------------------------------------------------------


100%|██████████| 98/98 [01:11<00:00,  1.37it/s]


Train Loss : 0.681
Validation Loss : 0.674
Validation Accuracy  : 0.583
----------------------------------------------------------------------------------------------------
EPOCH 3
----------------------------------------------------------------------------------------------------


...

100%|██████████| 98/98 [01:06<00:00,  1.47it/s]


Train Loss : 0.452
Validation Loss : 0.493
Validation Accuracy  : 0.774
----------------------------------------------------------------------------------------------------
EPOCH 10
----------------------------------------------------------------------------------------------------


100%|██████████| 98/98 [01:13<00:00,  1.33it/s]

Train Loss : 0.442
Validation Loss : 0.433
Validation Accuracy  : 0.815


([tensor(0.6892),
  tensor(0.6743),
  tensor(0.5298),
  tensor(0.4684),
  tensor(0.4693),
  tensor(0.4564),
  tensor(0.4491),
  tensor(0.4000),
  tensor(0.4929),
  tensor(0.4326)],
 [0.5535714285714286,
  0.5833333333333334,
  0.8035714285714286,
  0.7976190476190477,
  0.7916666666666666,
  0.8095238095238095,
  0.8214285714285714,
  0.8392857142857143,
  0.7738095238095238,
  0.8154761904761905],
 [tensor(0.6892),
  tensor(0.6743),
  tensor(0.5298),
  tensor(0.4684),
  tensor(0.4693),
  tensor(0.4564),
  tensor(0.4491),
  tensor(0.4000),
  tensor(0.4929),
  tensor(0.4326)],
 [0.5535714285714286,
  0.5833333333333334,
  0.8035714285714286,
  0.7976190476190477,
  0.7916666666666666,
  0.8095238095238095,
  0.8214285714285714,
  0.8392857142857143,
  0.7738095238095238,
  0.8154761904761905])

In [136]:
text_classifier_10_epochs.eval()
with torch.no_grad():
    print('Predicted values with only 10 epochs: ')
    print(torch.sigmoid(text_classifier_10_epochs(test_x[:10].to(device))))
    print(test_y[:10])
    print(f'The Accuracy of our model with 10 epochs is of {evaluateModel(text_classifier_10_epochs, loss_fn, test_x, test_y)}')

Predicted values with only 10 epochs: 
tensor([[0.2538],
        [0.2538],
        [0.9969],
        [0.2538],
        [0.2538],
        [0.8181],
        [0.9966],
        [0.2538],
        [0.2451],
        [0.2538]])
tensor([0, 0, 1, 0, 0, 1, 1, 0, 0, 1])
The Accuracy of our model with 10 epochs is of 0.7661


### Now let's use the next cell for hyper-parameter tuning: 

In [173]:
epochs = 10
learning_rate = 1e-3

loss_fn = nn.BCEWithLogitsLoss()
text_classifier_10_epochs_5_layers_10_hiddendim = LSTM(20,1, 2).to(device)

optimizer = RMSprop(text_classifier_10_epochs_5_layers_10_hiddendim.parameters(), lr=learning_rate)

print("STARTING TRAINING")
print("MODEL ARCHITECTURE:")
print(text_classifier_10_epochs_5_layers_10_hiddendim)
print(" ")


TrainModel(text_classifier_10_epochs_5_layers_10_hiddendim, loss_fn, optimizer, train_loader, epochs)

STARTING TRAINING
MODEL ARCHITECTURE:
LSTM(
  (embedding): Embedding(20000, 50, padding_idx=0)
  (lstm): LSTM(50, 20, num_layers=2, batch_first=True)
  (linear): Linear(in_features=20, out_features=100, bias=True)
  (relu): ReLU()
  (fc): Linear(in_features=100, out_features=1, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
)
 
----------------------------------------------------------------------------------------------------
EPOCH 1
----------------------------------------------------------------------------------------------------


100%|██████████| 98/98 [01:14<00:00,  1.32it/s]


Train Loss : 0.694
Validation Loss : 0.685
Validation Accuracy  : 0.613
----------------------------------------------------------------------------------------------------
EPOCH 2
----------------------------------------------------------------------------------------------------


100%|██████████| 98/98 [01:10<00:00,  1.40it/s]


Train Loss : 0.683
Validation Loss : 0.665
Validation Accuracy  : 0.542
----------------------------------------------------------------------------------------------------
EPOCH 3
----------------------------------------------------------------------------------------------------


...

Train Loss : 0.622
Validation Loss : 0.584
Validation Accuracy  : 0.613
----------------------------------------------------------------------------------------------------
EPOCH 5
----------------------------------------------------------------------------------------------------


100%|██████████| 98/98 [01:09<00:00,  1.40it/s]


Train Loss : 0.604
Validation Loss : 0.579
Validation Accuracy  : 0.601
----------------------------------------------------------------------------------------------------
EPOCH 6
----------------------------------------------------------------------------------------------------


 79%|███████▊  | 77/98 [00:58<00:15,  1.32it/s]


KeyboardInterrupt: ignored

In [171]:
text_classifier_10_epochs_5_layers_10_hiddendim.eval()
with torch.no_grad():
    print('Predicted values with only 10 epochs: ')
    print(torch.sigmoid(text_classifier_10_epochs_5_layers_10_hiddendim(test_x[:10].to(device))))
    print(test_y[:10])
    print(f'The Accuracy of our model with 2 epochs and 10 layers is of {evaluateModel(text_classifier_10_epochs_5_layers_10_hiddendim, loss_fn, test_x, test_y)}')

Predicted values with only 10 epochs: 
tensor([[0.4823],
        [0.4823],
        [0.9953],
        [0.4823],
        [0.4823],
        [0.4823],
        [0.0402],
        [0.4823],
        [0.4812],
        [0.4823]])
tensor([0, 0, 1, 0, 0, 1, 1, 0, 0, 1])
The Accuracy of our model with 2 epochs and 10 layers is of 0.56395


-0.53915 with 2 epochs and 20 hidden dim and 2 lstm layers

-0.54175 with 2 epochs and 20 hidden dim and 3 lstm layers

-0.5472 with 2 epochs and 20 hidden dim and 5 lstm layers

-0.61775 with 2 epochs and 40 hidden dim and 5 lstm layers

-0.50055 with 10 epochs and 40 hidden dim and 5 lstm layers